## Lab 1

### Data Quality

The following feautre have signficant NA values that must be handled prior to further analysis.

* *Age*
* *Height*
* *Weight*
* *Medal*

The first area of concern with the overall quality of the data is the number of NAs for the *height* and *weight* features (22.19% and 23.19% respectively).  This is exacerbated by the fact that a simple median value imputation will not provide a holistic solution since median height and weight values vary significantly per sport.  The most populous NA feature is *medal* (85.33%), however its solution is the simplest.  It can safely be assumed that this is not a mistake since there are three medals awarded per each event per Olympics, so a large percentage of NA values is this column is expcected. I replaced NA with *'No Medal'*. 

In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
df = pd.read_csv('~/olympics/data/athlete_events.csv')
df.isnull().sum()

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64

In [174]:
agena = (df['Age'].isnull().sum()/len(df['Age'])) * 100
heightna = (df['Height'].isnull().sum()/len(df['Height'])) * 100
weightna = (df['Weight'].isnull().sum()/len(df['Height'])) * 100
medalna = (df['Medal'].isnull().sum()/len(df['Medal'])) * 100
print('Weight NA percentage: %.2f' % (agena) + '%')
print('Height NA percentage: %.2f' % (heightna) + '%')
print('Weight NA percentage: %.2f' % (weightna) + '%')
print('Medal NA percentage: %.2f' % (medalna) + '%')

Weight NA percentage: 3.49%
Height NA percentage: 22.19%
Weight NA percentage: 23.19%
Medal NA percentage: 85.33%


In [175]:
# Fill all Medal NAs with 'No Medal'
df.Medal = df.Medal.fillna('No Medal')
#list of sports with no Height/Weight listed and competed since 1936
oldsports = ['Aeronautics', 'Alpinism', 'Art Competitions','Basque Pelota', 'Cricket', 'Croquet', 'Jeu De Paume','Military Ski Patrol',
             'Motorboating','Polo', 'Racquets', 'Roque']
df['Sport'].value_counts()
#drop the 611 observations containing these sports
df = df.drop(df[df.Sport.isin(oldsports)].index)
# Impute NA values by Sex and Sport
df.Height = df.groupby(['Sex', 'Sport'])['Height'].apply(lambda x: x.fillna(x.mean()))
df.Weight = df.groupby(['Sex', 'Sport'])['Weight'].apply(lambda x: x.fillna(x.mean()))
df.Age = df.groupby(['Sex', 'Sport'])['Age'].apply(lambda x: x.fillna(x.mean()))
df.groupby(['Sex','Sport']).describe()

Age                                    \
                                 count       mean       std   min    25%   
Sex Sport                                                                  
F   Alpine Skiing               3398.0  22.334609  3.772394  14.0  20.00   
    Archery                     1015.0  26.508458  7.603019  14.0  21.00   
    Athletics                  11666.0  24.933574  4.654056  12.0  22.00   
    Badminton                    740.0  25.047297  3.942644  16.0  22.00   
    Basketball                  1256.0  25.517516  4.062619  16.0  22.00   
    Beach Volleyball             276.0  28.315217  4.133049  18.0  25.00   
    Biathlon                    1863.0  25.799249  4.050362  17.0  23.00   
    Bobsleigh                    143.0  27.832168  4.087056  19.0  25.00   
    Boxing                        72.0  26.638889  4.230358  17.0  24.00   
    Canoeing                    1380.0  25.337446  4.757658  15.0  22.00   
    Cross Country Skiing        3385.0  25.655244  4.428332  15.0  22.00   
    Curling                      222.0  29.972973  6.908529  17.0  24.25   
    Cycling                     1394.0  26.899570  5.207258  17.0  23.00   
    Diving                      1210.0  21.517677  4.168557  12.0  18.00   
    Equestrianism               1246.0  34.217846  9.009052  16.0  27.00   
    Fencing                     2000.0  26.341105  5.206464  14.0  23.00   
    Figure Skating              1172.0  20.812554  4.530487  11.0  17.00   
    Football                    1012.0  24.909091  4.090185  16.0  22.00   
    Freestyle Skiing             433.0  24.330254  4.820811  13.0  21.00   
    Golf                          70.0  27.338235  6.254529  18.0  23.00   
    Gymnastics                  9129.0  19.232751  3.982255  11.0  16.00   
    Handball                    1401.0  25.883922  4.314619  15.0  23.00   
    Hockey                      1459.0  25.332419  3.770670  15.0  23.00   
    Ice Hockey                   754.0  24.055703  4.496370  15.0  21.00   
    Judo                        1093.0  25.163769  4.054311  14.0  22.00   
    Luge                         377.0  23.649867  5.002069  15.0  20.00   
    Modern Pentathlon            164.0  25.524390  4.777826  15.0  22.00   
    Rhythmic Gymnastics          658.0  18.737082  2.505250  13.0  17.00   
    Rowing                      2193.0  25.402645  4.385894  14.0  22.00   
    Rugby Sevens                 148.0  26.114865  3.985558  18.0  23.75   
...                                ...        ...       ...   ...    ...   
M   Gymnastics                 17578.0  24.653482  4.116102  10.0  22.00   
    Handball                    2264.0  26.569902  3.988745  14.0  24.00   
    Hockey                      3958.0  25.866276  4.021011  16.0  23.00   
    Ice Hockey                  4762.0  26.114304  4.351379  16.0  23.00   
    Judo                        2708.0  25.397378  3.833503  16.0  23.00   
    Lacrosse                      60.0  26.861111  3.800771  14.0  25.00   
    Luge                        1102.0  24.960909  4.796619  16.0  22.00   
    Modern Pentathlon           1513.0  26.485616  4.013098  16.0  24.00   
    Nordic Combined             1344.0  24.342322  3.897090  15.0  22.00   
    Rowing                      8402.0  25.241112  4.291308  11.0  22.00   
    Rugby                        162.0  24.385965  2.937250  18.0  23.00   
    Rugby Sevens                 151.0  25.980132  3.563650  19.0  23.00   
    Sailing                     5660.0  30.906622  8.004672  13.0  25.00   
    Shooting                    9724.0  34.254351  8.140460  15.0  28.00   
    Short Track Speed Skating    773.0  23.141009  3.511939  15.0  20.00   
    Skeleton                     133.0  30.977099  6.827380  18.0  26.00   
    Ski Jumping                 2371.0  23.373780  4.048217  14.0  20.00   
    Snowboarding                 520.0  24.907692  4.593614  15.0  22.00   
    Speed Skating               3532.0  24.633646  4.082396  14.0  22.00   
    Swi

In [176]:
df.isnull().sum()
unique_teams = df.Team.unique()
# Horse names used in Team column for Equestrian Events
print(unique_teams)
dfregions = pd.read_csv('~/olympics/data/noc_regions.csv')
# Merge athletes and regions dataframes to clean country names
df = pd.merge(left = dfregions, right = df, how = 'left', on=['NOC'])
print(df.tail())

['China' 'Denmark' 'Denmark/Sweden' ..., 'Solos Carex' 'Dow Jones' 'Digby']
        NOC    region notes        ID                   Name Sex   Age  \
266952  ZIM  Zimbabwe   NaN  130832.0         Hillary Wilson   F  15.0   
266953  ZIM  Zimbabwe   NaN  130832.0         Hillary Wilson   F  15.0   
266954  ZIM  Zimbabwe   NaN  130880.0    Peter Arthur Wilson   M  20.0   
266955  ZIM  Zimbabwe   NaN  131478.0  Jennifer "Jenny" Wood   F  16.0   
266956  ZIM  Zimbabwe   NaN  135497.0          Lloyd Zvasiya   M  23.0   

        Height  Weight      Team        Games    Year  Season    City  \
266952   157.0    65.0  Zimbabwe  1960 Summer  1960.0  Summer    Roma   
266953   157.0    65.0  Zimbabwe  1960 Summer  1960.0  Summer    Roma   
266954   185.0    80.0  Zimbabwe  1980 Summer  1980.0  Summer  Moskva   
266955   172.0    64.0  Zimbabwe  1964 Summer  1964.0  Summer   Tokyo   
266956   190.0    80.0  Zimbabwe  2004 Summer  2004.0  Summer  Athina   

            Sport                       

In [177]:
df.rename(index = str, columns = {'region' : 'country'}, inplace = True)
df.drop(columns = ['notes'])

,NOC,country,ID,Name,Sex,Age,Height,Weight,Team,Games,Year,Season,City,Sport,Event,Medal
0,AFG,Afghanistan,502.0,Ahmad Shah Abouwi,M,25.866276,176.852577,73.343761,Afghanistan,1956 Summer,1956.0,Summer,Melbourne,Hockey,Hockey Men's Hockey,No Medal
1,AFG,Afghanistan,1076.0,Jammal-ud-Din Affendi,M,28.000000,176.852577,73.343761,Afghanistan,1936 Summer,1936.0,Summer,Berlin,Hockey,Hockey Men's Hockey,No Medal
2,AFG,Afghanistan,1101.0,Mohammad Anwar Afzal,M,23.463606,177.480339,73.086644,Afghanistan,1948 Summer,1948.0,Summer,London,Football,Football Men's Football,No Medal
3,AFG,Afghanistan,1745.0,Mohammad Aktar,M,17.000000,156.000000,48.000000,Afghanistan,1980 Summer,1980.0,Summer,Moskva,Wrestling,"Wrestling Men's Light-Flyweight, Freestyle",No Medal
4,AFG,Afghanistan,4628.0,Mohammad Daoud Anwary,M,22.000000,172.870686,76.400640,Afghanistan,1964 Summer,1964.0,Summer,Tokyo,Wrestling,"Wrestling Men's Bantamweight, Freestyle",No Medal
5,AFG,Afghanistan,5285.0,Mohammad Arref,M,25.821827,172.870686,52.000000,Afghanistan,1972 Summer,1972.0,Summer,Munich,Wrestling,"Wrestling Men's Flyweight, Freestyle",No Medal
6,AFG,Afghanistan,5582.0,Mohammad Asif,M,17.000000,176.852577,73.343761,Afghanistan,1936 Summer,1936.0,Summer,Berlin,Hockey,Hockey Men's Hockey,No Medal
7,AFG,Afghanistan,5678.0,Abdul Ghafoor Assar,M,23.463606,177.480339,73.086644,Afghanistan,1948 Summer,1948.0,Summer,London,Football,Football Men's Football,No Medal
8,AFG,Afghanistan,5679.0,Abdul Ghani Assar,M,23.463606,177.480339,73.086644,Afghanistan,1948 Summer,1948.0,Summer,London,Football,Football Men's Football,No Medal
9,AFG,Afghanistan,5841.0,Sayed Ali Atta,M,22.000000,176.852577,73.343761,Afghanistan,1936 Summer,1936.0,Summer,Berlin,Hockey,Hockey Men's Hockey,No Medal


In [178]:
unique_countries = df.country.unique()
print(len(unique_countries))
print(unique_countries)

207
['Afghanistan' 'Curacao' 'Albania' 'Algeria' 'Andorra' 'Angola' 'Antigua'
 'Australia' 'Argentina' 'Armenia' 'Aruba' 'American Samoa' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bangladesh' 'Barbados' 'Burundi' 'Belgium' 'Benin'
 'Bermuda' 'Bhutan' 'Bosnia and Herzegovina' 'Belize' 'Belarus'
 'Czech Republic' 'Boliva' 'Botswana' 'Brazil' 'Bahrain' 'Brunei'
 'Bulgaria' 'Burkina Faso' 'Central African Republic' 'Cambodia' 'Canada'
 'Cayman Islands' 'Republic of Congo' 'Chad' 'Chile' 'China' 'Ivory Coast'
 'Cameroon' 'Democratic Republic of the Congo' 'Cook Islands' 'Colombia'
 'Comoros' 'Cape Verde' 'Costa Rica' 'Croatia' 'Greece' 'Cuba' 'Cyprus'
 'Denmark' 'Djibouti' 'Dominica' 'Dominican Republic' 'Ecuador' 'Egypt'
 'Eritrea' 'El Salvador' 'Spain' 'Estonia' 'Ethiopia' 'Russia' 'Fiji'
 'Finland' 'France' 'Germany' 'Micronesia' 'Gabon' 'Gambia' 'UK'
 'Guinea-Bissau' 'Georgia' 'Equatorial Guinea' 'Ghana' 'Grenada'
 'Guatemala' 'Guinea' 'Guam' 'Guyana' 'Haiti' 'Honduras' 'Hungary'
 'Indonesia' 

In [179]:
#Extract 'Event' Men's/Women's from the Event Column
df['Event'] = df['Event'].str.replace("(\w+\s+\w+\'s)", '')
print(df.tail())

        NOC   country notes        ID                   Name Sex   Age  \
266952  ZIM  Zimbabwe   NaN  130832.0         Hillary Wilson   F  15.0   
266953  ZIM  Zimbabwe   NaN  130832.0         Hillary Wilson   F  15.0   
266954  ZIM  Zimbabwe   NaN  130880.0    Peter Arthur Wilson   M  20.0   
266955  ZIM  Zimbabwe   NaN  131478.0  Jennifer "Jenny" Wood   F  16.0   
266956  ZIM  Zimbabwe   NaN  135497.0          Lloyd Zvasiya   M  23.0   

        Height  Weight      Team        Games    Year  Season    City  \
266952   157.0    65.0  Zimbabwe  1960 Summer  1960.0  Summer    Roma   
266953   157.0    65.0  Zimbabwe  1960 Summer  1960.0  Summer    Roma   
266954   185.0    80.0  Zimbabwe  1980 Summer  1980.0  Summer  Moskva   
266955   172.0    64.0  Zimbabwe  1964 Summer  1964.0  Summer   Tokyo   
266956   190.0    80.0  Zimbabwe  2004 Summer  2004.0  Summer  Athina   

            Sport                            Event     Medal  
266952   Swimming             100 metres Butterfly  N

In [180]:
oldolympians = pd.DataFrame(df[df.Age >= 72])
print(oldolympians)
youngolympians = pd.DataFrame(df[df.Age <= 11])
print(youngolympians)
tallestolympians = pd.DataFrame(df[df.Height >= 220])
print(tallestolympians)
shortestolympians = pd.DataFrame(df[df.Height <= 130])
print(shortestolympians)
heaviestolympians = pd.DataFrame(df[df.Weight >= 200])
print(heaviestolympians)
lightestolympians = pd.DataFrame(df[df.Weight <= 25])
print(lightestolympians)

        NOC  country notes        ID  \
17225   AUT  Austria   NaN  127395.0   
17226   AUT  Austria   NaN  127395.0   
224607  SWE   Sweden   NaN  117046.0   
224608  SWE   Sweden   NaN  117046.0   
224609  SWE   Sweden   NaN  117046.0   

                                             Name Sex   Age      Height  \
17225   Arthur von Pongracz de Szent-Mikls und vr   M  72.0  176.563172   
17226   Arthur von Pongracz de Szent-Mikls und vr   M  72.0  176.563172   
224607                          Oscar Gomer Swahn   M  72.0  175.977163   
224608                          Oscar Gomer Swahn   M  72.0  175.977163   
224609                          Oscar Gomer Swahn   M  72.0  175.977163   

           Weight     Team        Games    Year  Season       City  \
17225   70.924559  Austria  1936 Summer  1936.0  Summer     Berlin   
17226   70.924559  Austria  1936 Summer  1936.0  Summer     Berlin   
224607  77.834960   Sweden  1920 Summer  1920.0  Summer  Antwerpen   
224608  77.834960   Sweden  

## Exceptional Work

In [193]:
grouped = df.groupby(['country', 'Medal']).size().to_frame(name = 'count').reset_index()
groupedpivot = grouped.pivot_table(index = ['country'], columns = 'Medal')
groupedpivot.fillna(0, inplace = True)
print(groupedpivot.head())
groupedpivot.to_excel('MedalCounts.xls')

                count                     
Medal          Bronze Gold No Medal Silver
country                                   
Afghanistan       2.0  0.0    124.0    0.0
Albania           0.0  0.0     70.0    0.0
Algeria           8.0  5.0    534.0    4.0
American Samoa    0.0  0.0     37.0    0.0
Andorra           0.0  0.0    169.0    0.0


In [186]:
%%HTML
<script type='text/javascript' src='https://us-east-1.online.tableau.com/javascripts/api/viz_v1.js'></script><div class='tableauPlaceholder' style='width: 1680px; height: 784px;'><object class='tableauViz' width='1680' height='784' style='display:none;'><param name='host_url' value='https%3A%2F%2Fus-east-1.online.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='&#47;t&#47;smudatascience' /><param name='name' value='Olympics&#47;Sheet1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='showAppBanner' value='false' /><param name='filter' value='iframeSizedToWindow=true' /></object></div>

In [197]:
grouped1 = df.groupby('country')
groupedheight = grouped1['Height']
groupedheight = pd.DataFrame(groupedheight.agg('max'))
print(groupedheight.head())
groupedheight.to_excel('MaxHeights.xls')

                Height
country               
Afghanistan      186.0
Albania          195.0
Algeria          200.0
American Samoa   188.0
Andorra          189.0


In [198]:
%%HTML
<script type='text/javascript' src='https://us-east-1.online.tableau.com/javascripts/api/viz_v1.js'></script><div class='tableauPlaceholder' style='width: 1680px; height: 784px;'><object class='tableauViz' width='1680' height='784' style='display:none;'><param name='host_url' value='https%3A%2F%2Fus-east-1.online.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='&#47;t&#47;smudatascience' /><param name='name' value='Olympics1&#47;Sheet2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='showAppBanner' value='false' /><param name='filter' value='iframeSizedToWindow=true' /></object></div>

In [200]:
grouped2 = df.groupby('country')
groupedweight = grouped2['Weight']
groupedweight = pd.DataFrame(groupedweight.agg('max'))
print(groupedweight.head())
groupedweight.to_excel('MaxWeights.xls')

                Weight
country               
Afghanistan      111.0
Albania          130.0
Algeria          125.0
American Samoa   127.0
Andorra           90.0


In [202]:
%%HTML
<script type='text/javascript' src='https://us-east-1.online.tableau.com/javascripts/api/viz_v1.js'></script><div class='tableauPlaceholder' style='width: 1680px; height: 784px;'><object class='tableauViz' width='1680' height='784' style='display:none;'><param name='host_url' value='https%3A%2F%2Fus-east-1.online.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='&#47;t&#47;smudatascience' /><param name='name' value='Olympics2&#47;Sheet3' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='showAppBanner' value='false' /><param name='filter' value='iframeSizedToWindow=true' /></object></div>